In [5]:
# ======================================
# QA MODE - Kriging for 1–2 hours only
# ======================================
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pykrige.ok import OrdinaryKriging
import os

# ---------------- Paths
CSV_FILE  = r"C:\Users\krish\Desktop\SpatialCARE\Hourly\pasig_hourly_corrected.csv"
PASIG_SHP = r"C:\Users\krish\Desktop\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
OUT_DIR   = r"C:\Users\krish\Desktop\SpatialCARE\Hourly\Outputs\Kriging_QA"
os.makedirs(OUT_DIR, exist_ok=True)

# ---------------- Load data
df = pd.read_csv(CSV_FILE)
df["datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"])
df = df.set_index("datetime")

# Load Pasig shapefile & force to EPSG:4326 (lon/lat)
pasig = gpd.read_file(PASIG_SHP)
pasig = pasig.to_crs("EPSG:4326")
bounds = pasig.total_bounds  # xmin, ymin, xmax, ymax

# ---------------- QA timestamps (pick 1–2 manually)
qa_times = [
    "2025-03-01 00:00:00",
    "2025-03-01 12:00:00"
]

for ts in qa_times:
    try:
        g = df.loc[ts]   # station data for this hour
    except KeyError:
        print(f"⚠️ No data found for {ts}")
        continue

    if g.empty: 
        continue

    # Ensure DataFrame (not Series)
    g = g if isinstance(g, pd.DataFrame) else g.to_frame().T

    # Grid over Pasig bounds
    grid_lon = np.linspace(bounds[0], bounds[2], 200)
    grid_lat = np.linspace(bounds[1], bounds[3], 200)

    # Kriging input
    values = g["pm25"].values
    lons = g["longitude"].values
    lats = g["latitude"].values

    if len(values) < 3:
        print(f"⚠️ Not enough stations for {ts} (need ≥3)")
        continue

    OK = OrdinaryKriging(
        lons, lats, values,
        variogram_model="spherical",
        verbose=False, enable_plotting=False
    )
    z, ss = OK.execute("grid", grid_lon, grid_lat)

    # ---------------- Plot
    fig, ax = plt.subplots(figsize=(7,7))

    # Mask invalid values (NaN from outside convex hull)
    z_masked = np.ma.masked_invalid(z)

    # Use contourf for smoother kriging surface
    cax = ax.contourf(grid_lon, grid_lat, z_masked, 
                      levels=20, cmap="jet", alpha=0.7)

    # Overlay Pasig boundary
    pasig.boundary.plot(ax=ax, color="black", linewidth=0.5, zorder=3)

    # Plot stations
    ax.scatter(lons, lats, c="k", s=40, label="Stations", zorder=4)

    fig.colorbar(cax, ax=ax, label="PM₂.₅ (µg/m³)")
    ax.set_title(f"Kriging Interpolation - {ts}")
    ax.legend()

    # Zoom to Pasig boundary
    ax.set_xlim(bounds[0], bounds[2])
    ax.set_ylim(bounds[1], bounds[3])

    out_file = os.path.join(OUT_DIR, f"kriging_{ts.replace(':','-').replace(' ','_')}.png")
    plt.savefig(out_file, dpi=150, bbox_inches="tight")
    plt.close()
    print(f"✅ Saved QA plot: {out_file}")


✅ Saved QA plot: C:\Users\krish\Desktop\SpatialCARE\Hourly\Outputs\Kriging_QA\kriging_2025-03-01_00-00-00.png
✅ Saved QA plot: C:\Users\krish\Desktop\SpatialCARE\Hourly\Outputs\Kriging_QA\kriging_2025-03-01_12-00-00.png
